In [8]:
model_name = "llama2"

In [9]:
from langchain_community.llms import Ollama
llm = Ollama(model=model_name)

In [11]:
llm.invoke("What is the meaning of life?")

"\nThe meaning of life is a philosophical and existential question that has been debated throughout human history. It's a complex and multi-faceted topic, and there is no one definitive answer. However, here are some possible meanings of life:\n\n1. Survival and Self-Preservation: At its most basic level, the meaning of life can be seen as the drive to survive and preserve oneself. This is evident in the natural world, where living beings must fight for resources and struggle to stay alive in a hostile environment.\n2. Personal Fulfillment: Many people believe that the meaning of life is to find personal fulfillment and happiness. This can be achieved through pursuing one's passions, building meaningful relationships, and cultivating a sense of purpose and direction.\n3. Social Connection and Community: For many individuals, the meaning of life is closely tied to social connections and community. People may find meaning in their relationships with family, friends, and colleagues, as we

In [12]:
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are world class technical documentation writer."),
    ("user", "{input}")
])

In [13]:
chain = prompt | llm 

In [14]:
chain.invoke({"input": "What is the meaning of life?"}) # MUST BE A DICT

"\nAh, a question that has puzzled philosophers and theologians for centuries! As a world-class technical documentation writer, I must say that the meaning of life is not just limited to a single definition or explanation. It's a complex and multifaceted concept that can vary from person to person, culture to culture, and even individual to individual.\n\nBut, as a technical writer, I believe that the meaning of life can be distilled into a few key points:\n\n1. Survival: The most basic and fundamental aspect of life is survival. Our bodies are made up of cells, organs, and systems that work together to keep us alive and functioning. From a technical perspective, this means that our body's functions must be properly maintained through proper nutrition, hydration, exercise, and rest.\n2. Growth: As living beings, we are constantly growing and developing. This can be seen in the way we learn new skills, gain knowledge, and evolve as individuals. Technically speaking, this means that our 

In [15]:
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

In [16]:
chain = prompt | llm | output_parser    

In [17]:
chain.invoke({"input": "What is the meaning of life?"})

"\nAh, a question that has puzzled philosophers and thinkers for centuries! As a technical documentation writer, I must say that the meaning of life is a complex and multifaceted concept that can have different interpretations depending on one's perspective. However, if I were to approach this question from a purely analytical standpoint, I would argue that the meaning of life can be broken down into several key components:\n\n1. Survival: The most basic and fundamental aspect of life is survival. Living organisms must constantly strive to maintain their existence and continue to evolve in order to ensure their continued survival.\n2. Growth and Development: As living beings, we have the innate desire to grow and develop. This can be seen in our physical bodies, as well as in our mental and emotional capacities. The pursuit of knowledge, wisdom, and personal growth are all part of this drive for development.\n3. Relationships: Human beings are social creatures, and the connections we f

In [18]:
from langchain_community.document_loaders import WebBaseLoader
loader = WebBaseLoader("https://docs.smith.langchain.com/overview")

docs = loader.load()

In [19]:
docs

[Document(page_content='\n\n\n\n\nLangSmith Overview and User Guide | 🦜️🛠️ LangSmith\n\n\n\n\n\nSkip to main content🦜️🛠️ LangSmith DocsPython DocsJS/TS DocsSearchGo to AppLangSmithOverviewTracingTesting & EvaluationOrganizationsHubLangSmith CookbookRelease NotesOverviewOn this pageLangSmith Overview and User GuideBuilding reliable LLM applications can be challenging. LangChain simplifies the initial setup, but there is still work needed to bring the performance of prompts, chains and agents up the level where they are reliable enough to be used in production.Over the past two months, we at LangChain have been building and using LangSmith with the goal of bridging this gap. This is our tactical user guide to outline effective ways to use LangSmith and maximize its benefits.On by default\u200bAt LangChain, all of us have LangSmith’s tracing running in the background by default. On the Python side, this is achieved by setting environment variables, which we establish whenever we launch a 

In [20]:
from langchain_community.embeddings import OllamaEmbeddings

embeddings = OllamaEmbeddings(model=model_name)

# build the index

In [21]:
from langchain_community.vectorstores import FAISS 
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [22]:
text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents(docs)
vector = FAISS.from_documents(documents, embeddings)

Now that we have this data indexed in a vectorstore, we will create a retrieval chain. This chain will take an incoming question, look up relevant documents, then pass those documents along with the original question into an LLM and ask it to answer the original question.

First, let's set up the chain that takes a question and the retrieved documents and generates an answer.

In [23]:
from langchain.chains.combine_documents import create_stuff_documents_chain

prompt = ChatPromptTemplate.from_template("""
            Answer the following question based only on the provided context:
            <context>
            {context}
            </context>
            
            Question: {input}
            """)

document_chain = create_stuff_documents_chain(llm, prompt)

If we wanted to, we could run this ourselves by passing in documents directly:

In [24]:
from langchain_core.documents import Document

document_chain.invoke({
    "input": "how can langsmith help with testing?",
    "context": [Document(page_content="langsmith can help you visualize test results")]
})

'Based on the provided context, Langsmith can help with testing by providing a visual representation of test results. This means that Langsmith can be used to create graphs, charts, or other visualizations of test data, which can help developers and testers understand the results of their tests more easily. By providing a visual representation of test data, Langsmith can make it easier to identify trends, patterns, and issues in the test results, which can inform decision-making and improve the testing process.'

However, we want the documents to first come from the retriever we just set up. That way, for a given question we can use the retriever to dynamically select the most relevant documents and pass those in.

In [25]:
from langchain.chains import create_retrieval_chain

retriever = vector.as_retriever()
retrieval_chain = create_retrieval_chain(retriever, document_chain)

In [26]:
response = retrieval_chain.invoke({
    "input": "how can langsmith help with testing?"
})

print(response['answer'])

LangSmith can assist with testing by providing several features to simplify the process:

1. **Running the chain over the data points**: LangSmith simplifies dataset uploading and allows running a chain over the data points. The output is visualized, and feedback can be assigned to the runs for manual review.
2. **Evaluating changes to a prompt or chain**: LangSmith offers automatic evaluation metrics, but human review is still necessary for quality and reliability. LangSmith makes it easy to manually review and annotate runs through annotation queues.
3. **Monitoring the application in production**: LangSmith can be used to monitor the application in much the same way as debugging. Logs, visualization of latency and token usage statistics, and troubleshooting specific issues are possible. Each run can be assigned string tags or key-value metadata, allowing for filtering and analysis.
4. **Exporting datasets**: LangSmith makes it easy to curate datasets, which can be exported for use i

## Conversational Retrieval Chain

The chain we've created so far can only answer single questions. One of the main types of LLM applications that people are building are chat bots. So how do we turn this chain into one that can answer follow up questions?

We can still use the `create_retrieval_chain` function, but we need to change two things:

1. The retrieval method should now not just work on the most recent input, but rather should take the whole history into account.
2. The final LLM chain should likewise take the **whole history** into account


#### Updating Retrieval

In order to update retrieval, we will create a new chain. This chain will take in the most recent input (`input`) and the conversation history (`chat_history`) and use an LLM to generate a search query.

In [27]:
from langchain.chains import create_history_aware_retriever 
from langchain.prompts import MessagesPlaceholder

# First we need a prompt that we can pass into an LM to generate this search query 

prompt = ChatPromptTemplate.from_messages([
     MessagesPlaceholder(variable_name="chat_history"),
     ("user", "{input}"),
     ("user", "Given the previous conversation, generate a search query to look up in order to get the information relevant to the conversation.")
     
 ])

retriever_chain = create_history_aware_retriever(llm=llm, retriever=retriever, prompt=prompt)

We can test this out by passing in an instance where the user is asking a follow up question.

In [28]:
from langchain_core.messages import HumanMessage, AIMessage

chat_history = [HumanMessage(content="Can LangSmith help me test my LLM applications?"), 
                AIMessage(content="Yes, LangSmith can help you visualize test results")]
retriever_chain.invoke({"chat_history": chat_history,
                "input": "tell me how"})

[Document(page_content="Skip to main content🦜️🛠️ LangSmith DocsPython DocsJS/TS DocsSearchGo to AppLangSmithOverviewTracingTesting & EvaluationOrganizationsHubLangSmith CookbookRelease NotesOverviewOn this pageLangSmith Overview and User GuideBuilding reliable LLM applications can be challenging. LangChain simplifies the initial setup, but there is still work needed to bring the performance of prompts, chains and agents up the level where they are reliable enough to be used in production.Over the past two months, we at LangChain have been building and using LangSmith with the goal of bridging this gap. This is our tactical user guide to outline effective ways to use LangSmith and maximize its benefits.On by default\u200bAt LangChain, all of us have LangSmith’s tracing running in the background by default. On the Python side, this is achieved by setting environment variables, which we establish whenever we launch a virtual environment or open our bash shell and leave them set. The same 

You should see that this returns documents about testing in LangSmith. This is because the LLM generated a new query, combining the chat history with the follow up question.

Now that we have this new retriever, we can create a new chain to continue the conversation with these retrieved documents in mind.

# Agent

We've so far create examples of chains - where each step is known ahead of time. The final thing we will create is an agent - where the LLM decides what steps to take.

One of the first things to do when building an agent is to decide what tools it should have access to. For this example, we will give the agent access two tools:

1. The retriever we just created. This will let it easily answer questions about LangSmith
2. A search tool. This will let it easily answer questions that require up to date information.

First, let's set up a tool for the retriever we just created:

In [29]:
from langchain.tools.retriever import create_retriever_tool

retriever_tool = create_retriever_tool(
    retriever,
    "langsmith_search",
    "Search for information about LangSmith. For any questions about LangSmith, you must use this tool!",
)

In [30]:
import getpass
import os
# from dotenv import load_dotenv

In [4]:
# TAVILY_API_KEY = getpass.getpass("Enter your Tavily API key: ")

In [ ]:
# load_dotenv()
# OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [ ]:
from langchain_community.tools.tavily_search import TavilySearchResults

search = TavilySearchResults()

In [ ]:
search

TavilySearchResults()

In [ ]:
tools = [retriever_tool, search]

NameError: name 'retriever_tool' is not defined